In [ ]:
# import package
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from pyimagesearch.preprocessing import ImageToArrayPreprocessor
from pyimagesearch.preprocessing import AspectAwarePreprocessor
from pyimagesearch.datasets import SimpleDatasetLoader
from pyimagesearch.nn.conv import FCHeadNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from imutils import paths
import numpy as np 
import os 

In [ ]:
args = {
    'dataset' : '',
    'model' : '',
}

In [ ]:
# construct the image data generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [ ]:
# grab the list of image that we'll be describing, then extract the class label names from the image path
print("[info] loading images...")
imagePaths = list(paths.list_images(args['dataset']))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

In [ ]:
# init the image preprocessor
aap = AspectAwarePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixel intensifies to the range
# [0, 1]

sdl = SimpleDatasetLoader(preprocessors=(aap, iap))
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining for testing

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# convert the label from integer to vector
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit(testY)

In [ ]:
# load the VGG 16 network, ensuring the head FC layers sets are left off
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

# init the new head of the network, a set of FC layers followed by softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames), 256)

# place the head FC model on top of the base model -- this will become the actual model we will train
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
# loop over all layers in the base model and freeze them so they will not be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
# compile our model (this needs to be done after our setting our layers to be non-trainable)
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# train the head of the network for a few epochs (all other layers are frozen) --
# this will allow the new FC layers to start to become init with actual learned values
# versus pure random

print(["[INFO] training head...."])
model.fit_generator(aug.flow(trainX, trainY, batch_size=32), validation_data=(testX, testY), epochs=25, steps_per_epoch=len(trainX) // 32, verbose=1)

In [ ]:
# evaluate network after init
print("[INFO] evaluating after init...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classNames))

In [ ]:
# now that the head FC layers have been trained lets unfreeze the final set of conv layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True

In [ ]:
# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
print("[INFO] fine-tuning model...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
validation_data=(testX, testY), epochs=100,
steps_per_epoch=len(trainX) // 32, verbose=1)

In [ ]:
# evaluate the network on the fine-tuned model
print("[INFO] evaluating after fine-tuning...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1), target_names=classNames))


# save the model to disk
print("[INFO] serializing model...")
model.save(args["model"])